## Filtrado de datos relevantes

In [1]:
import pandas as pd
df=pd.read_csv('../../rawdata/ctu-13/capture20110810.binetflow.labels.gz')

In [2]:
df=df[["SrcAddr","DstAddr","TotBytes","SrcBytes"]]
df["DstBytes"]=df["TotBytes"]-df["SrcBytes"]
df.drop('TotBytes',inplace=True, axis=1)
df

,SrcAddr,DstAddr,SrcBytes,DstBytes
0,212.50.71.179,147.32.84.229,413,462
1,84.13.246.132,147.32.84.229,75,60
2,217.163.21.35,147.32.86.194,60,60
3,83.3.77.74,147.32.85.5,120,60
4,74.89.223.204,147.32.84.229,1596,1260
...,...,...,...,...
2824631,147.32.86.92,147.32.80.9,79,129
2824632,58.165.41.84,147.32.84.229,75,464
2824633,147.32.84.171,78.191.168.43,74,0
2824634,93.79.39.15,147.32.84.229,460,60


## Asigna en cada comunicación la menor ip como el source address

In [3]:
def ip_read(ip_str):
    aux_str=ip_str 
    if aux_str.count(":")==5:#Para que ande ip_address para direcciones de 6 bytes
        ip=int(ipaddress.ip_address('00:00:'+ip_str))
    else:
        ip=int(ipaddress.ip_address(ip_str))
    return ip
    
    

In [4]:
import ipaddress
idx=df['SrcAddr'].transform(ip_read)>df['DstAddr'].transform(ip_read) #Indice de los valores en los que src>dst

aux=df[idx]['SrcBytes'].copy() #Reordena los bytes
df.loc[idx,'SrcBytes']=df[idx]['DstBytes']
df.loc[idx,'DstBytes']=aux

aux=df[idx]['SrcAddr'].copy()  #Reordena los address
df.loc[idx,'SrcAddr']=df[idx]['DstAddr']
df.loc[idx,'DstAddr']=aux

## Agrupamiento comunicaciones que comparten emisor y receptor


In [5]:
df_gb=df.groupby(['SrcAddr','DstAddr'])
df_agg=df_gb.agg({
    'SrcBytes':'sum',
    'DstBytes':'sum'
})
df=df_agg.reset_index()


In [6]:
df

,SrcAddr,DstAddr,SrcBytes,DstBytes
0,0.0.0.0,255.255.255.255,743034,0
1,00:00:00:00:00:00,00:0c:29:97:a3:eb,1200,0
2,00:00:00:00:00:00,00:0c:29:9d:31:2f,1200,0
3,00:00:00:00:00:00,52:54:00:89:03:54,300,0
4,00:00:00:00:00:00,52:54:00:aa:cd:12,300,0
...,...,...,...,...
645863,fe80::7272:cfff:fe1b:d243,ff02::1:ffef:6697,86,0
645864,fe80::d036:46ef:ad35:fc09,ff02::1:3,504,0
645865,fe80::d036:46ef:ad35:fc09,ff02::1:ff1b:d243,2150,0
645866,fe80::f180:4f5c:7d4a:bbd7,ff02::1:3,1008,0
